<h3 style="text-align: center;"><b>«Практическое занятие по доверительным интервалам»</b></h3>

In [ ]:
import numpy as np
import scipy.stats as st
import pandas as pd
import math
import matplotlib.pyplot as plt

## Расчет по формуле

<b>Пример.</b> По данным $7$ измерений некоторой величины найдены средняя результатов
измерений, равная $30$ и выборочная дисперсия, равная $36$. Найдите границы, в которых с
надежностью $0,95$ заключено истинное значение измеряемой величины. 

In [ ]:
n = 7 # Размер выборки
dof = n - 1 # Число степеней свободы 
mn = 30 # Среднее значение
sigma = np.sqrt(36) # Стандартное отклонение
T = 2.45  # Параметр t распределения. Таблицу значений можно легко найти в поисковике

In [ ]:
(mn - T*sigma/np.sqrt(n), mn + T*sigma/np.sqrt(n))

## Собственная функция

Вспомогательная функция для подсчета доверительного интервала для маленьких выборок (до 30 элементов, t распределение).

In [ ]:
def mean_confidence_interval(data, confidence=0.95): 
    n = len(data)
    m, se = np.mean(data), st.sem(data)
    h = se * st.t.ppf((1 + confidence)/2, n-1)  #  функция ppf из scipy возвращает нам перцентиль
    return m-h, m, m+h 

In [ ]:
a =  [5,7,7,9,10,16,7,14,13,15,16,15,
     16,17,18,13,14,15,6,13, 14, 12 ,13 ]

print(mean_confidence_interval(a, 0.60))
print(mean_confidence_interval(a, 0.90))
print(mean_confidence_interval(a, 0.99))

<b>Пример.</b> Попробуем с помощью доверительного интервала оценить результаты выборов.

<p style="align: center;"><img align=center src="https://www.englisch-hilfen.de/images/usa/states/map.gif"  width=400></p>

In [ ]:
US_elect_county = 'https://raw.githubusercontent.com/a-milenkin/datasets_for_t-tests/main/2012_US_elect_county.csv'
 
df = pd.read_csv(US_elect_county, sep=',') # Откроем датасет

In [ ]:
df.dropna(how='any',inplace=True)  #Удалим пустые значения
df=df[df['%']!='#DIV/0!'][['State Postal','County Name','%']] #Удалим мусор из данных и оставим только нужное
df['%'] = df['%'].astype(float) # Приведем к численному значению результаты голосования
df.head()

In [ ]:
df['%'].hist(bins=15)
print(f"Среднее по всей выборке: {df['%'].mean():.3f}")

In [ ]:
some_states = ['AK','OH','WV','ME','CA','MD']

sub_df = df.loc[df['State Postal'].isin(some_states)]  # Выборка по нескольким штатам

l, s, r=mean_confidence_interval(sub_df['%'], 0.99)
l, s, r

## Встроенные функции scipy.stats

### Расчет с использованием нормального распределения

Длина выборки более 30 элементов.

<b>Пример.</b> Используем выборку размером $n=100$ и определим $90\%$ и $99\%$ доверительные интервалы, используя нормальное распределение.

In [ ]:
np.random.seed(54)  # Закрепим seed для воспроизводимости результата.

# Генерируем случайное множество с нормальным распределением, где среднее = loc, а стандартное отклонение = scale
dataN = st.norm.rvs(loc=25, scale=5, size=100)

In [ ]:
ci_lower, ci_upper = st.norm.interval(alpha=0.90, loc=np.mean(dataN), scale=st.sem(dataN))
print(f'Мы уверены на 90%, что истинное среднее значение генеральной совокупности находится в диапазоне между {ci_lower:5.3f} и {ci_upper:5.3f}')

In [ ]:
ci_lower, ci_upper = st.norm.interval(alpha=0.99, loc=np.mean(dataN), scale=st.sem(dataN))
print(f'Мы уверены на 99%, что истинное среднее значение генеральной совокупности находится в диапазоне между {ci_lower:5.3f} и {ci_upper:5.3f}')

### Расчет с использованием t распределения

Длина выборки менее 30 элементов. 

<b>Пример.</b> Используем выборку размером $n=20$ и определим $90\%$ и $99\%$ доверительные интервалы, используя t распределение.

In [ ]:
np.random.seed(54)  # Закрепим seed для воспроизводимости результата.

# Генерируем случайное множество с нормальным распределением, где среднее = loc, а стандартное отклонение = scale
dataT = st.norm.rvs(loc=25, scale=5, size=20)
dataT

In [ ]:
ci_lower, ci_upper = st.t.interval(alpha=0.90, df=len(dataT)-1, loc=np.mean(dataT), scale=st.sem(dataT))
print(f'Мы уверены на 90%, что истинное среднее значение генеральной совокупности находится в диапазоне между {ci_lower:4.3f} и {ci_upper:5.3f}')

In [ ]:
ci_lower, ci_upper = st.t.interval(alpha=0.99, df=len(dataT)-1, loc=np.mean(dataT), scale=st.sem(dataT))
print(f'Мы уверены на 99%, что истинное среднее значение генеральной совокупности находится в диапазоне между {ci_lower:4.3f} и {ci_upper:5.3f}')

## Определение размера выборки

<b>Пример.</b> Допустим, Вы журналист и оказались в 2016 году в преддверии выборов президента в США. Вы хотите по результатам своих исследований достаточно точно определить, кто победит на этих выборах. Сколько вам необходимо опросить людей, чтобы назвать будущего президента с точность в 95%?

Результаты предшествующих исследований свидетельствуют, что стандартное отклонение генеральной совокупности приблизительно равно 15. Пусть погрешность измерений $\Delta$ = 5. Таким образом, обозначим стандартное отклонение $\sigma$ = 15 и квантиль нормального распределения Z = 1,96.

$$ n = (\frac{Z*\sigma}{\Delta})^2$$
- $\Delta$ - допустимая ошибка/погрешность, 

- $\sigma$ - стандартное отклонение, 

- Z - квантиль нормального распределения


In [ ]:
sigma = 15  # Стандартное отклонение
Z = 1.96  # Квантиль нормального распределения. 
Delta = 5 # Погрешность

In [ ]:
N = int(np.ceil(((sigma*Z)/Delta)**2))
N

Сравним полученные результаты с генеральной совокупностью.

In [ ]:
print(f"Стандартное отклонение: {df['%'].std():.3f}")
print(f"Среднее: {df['%'].mean():.3f}")
df['%'].hist(bins=30);

In [ ]:
np.random.seed(67)  # Закрепим seed для воспроизводимости результата.

mn = df['%'].sample(n = int(35)).mean()  # Возьмем выборку из случайных n значений
print(f"Среднее выборки: {mn:.3f}")
print(f"Среднее генеральной совокупности: {df['%'].mean():.3f}")
print(f"Доверительный интервал: ({mn-Delta:.3f}, {mn+Delta:.3f})")

<b>Пример.</b> Часто возникают ситуации, когда размер выборки может зависеть от оценки размера генеральной совокупности. Например, если мы хотим провести опрос клиентов банка на предмет удобства или качества обслуживания. Неужели размер выборки будет одинаковый если у нас 10 000 клиентов или 1 000 000 клиентов?

В этом случае для определения размера выборки можно использовать оценку размера генеральной совокупности по следующей формуле.

In [ ]:
p = 0.8 # распределение голосов
Ddec = Delta/100.
Pop = 10000000 # Оценка размера генеральной совокупности

In [ ]:
int(np.ceil((Z**2*p*(1-p)/Ddec**2)/(1+(Z**2*p*(1-p)/(Pop*Ddec**2)))))